In [1]:
from kaggle_environments import make
from lux.game import Game
from lux.game_map import Cell, RESOURCE_TYPES, Position
from lux.game_objects import Unit
from lux.constants import Constants
from lux.game_constants import GAME_CONSTANTS
from lux import annotate
import math
import sys
import numpy as np
import random
from IPython.display import clear_output 
from lux.game import Game
from lux.game_map import Cell, RESOURCE_TYPES
from lux.constants import Constants
from lux.game_constants import GAME_CONSTANTS
from lux import annotate
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras import layers
from collections import deque
import random
import time

Loading environment football failed: No module named 'gfootball'


In [2]:
def get_inputs(game_state):
    # The shape of the map
    w,h = game_state.map.width, game_state.map.height
    # The map of ressources
    M = [ [0  if game_state.map.map[j][i].resource==None else game_state.map.map[j][i].resource.amount for i in range(w)]  for j in range(h)]
    
    M = np.array(M).reshape((h,w,1))
    
    # The map of units features
    U_player = [ [[0,0,0,0,0] for i in range(w)]  for j in range(h)]    
    units = game_state.players[0].units
    for i in units:
        U_player[i.pos.y][i.pos.x] = [i.type,i.cooldown,i.cargo.wood,i.cargo.coal,i.cargo.uranium]
    U_player = np.array(U_player)
    
    U_opponent = [ [[0,0,0,0,0] for i in range(w)]  for j in range(h)]
    units = game_state.players[1].units
    for i in units:
        U_opponent[i.pos.y][i.pos.x] = [i.type,i.cooldown,i.cargo.wood,i.cargo.coal,i.cargo.uranium]

    U_opponent = np.array(U_opponent)
    
    # The map of cities featrues
    e = game_state.players[0].cities
    C_player = [ [[0,0,0] for i in range(w)]  for j in range(h)]
    for k in e:
        citytiles = e[k].citytiles
        for i in citytiles:
            C_player[i.pos.y][i.pos.x] = [i.cooldown,e[k].fuel,e[k].light_upkeep]
    C_player = np.array(C_player)

    e = game_state.players[1].cities
    C_opponent = [ [[0,0,0] for i in range(w)]  for j in range(h)]
    for k in e:
        citytiles = e[k].citytiles
        for i in citytiles:
            C_opponent[i.pos.y][i.pos.x] = [i.cooldown,e[k].fuel,e[k].light_upkeep]
    C_opponent = np.array(C_opponent)
    
    # stacking all in one array
    E = np.dstack([M,U_opponent,U_player,C_opponent,C_player])
    return E

In [3]:
def get_model(s):
    inputs = keras.Input(shape=(s,s,17),name = 'TheGameMap')
    f = layers.Flatten()(inputs)   
    h,w= s,s
    f = layers.Dense(w*h,activation = "sigmoid")(f)
    f = layers.Reshape((h,w,-1))(f)
    units = layers.Dense(6,activation = "softmax",name = "Units_actions")(f)
    cities = layers.Dense(2,activation = "sigmoid",name = "Cities_actions")(f)
    output = layers.Concatenate()([units,cities])
    model = keras.Model(inputs = inputs, outputs = output)
    #model.compile(loss='mse', optimizer="adam")
    model.compile(loss='mse', optimizer="nadam")
    
    return model

In [4]:
model = get_model(12)
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 TheGameMap (InputLayer)        [(None, 12, 12, 17)  0           []                               
                                ]                                                                 
                                                                                                  
 flatten (Flatten)              (None, 2448)         0           ['TheGameMap[0][0]']             
                                                                                                  
 dense (Dense)                  (None, 144)          352656      ['flatten[0][0]']                
                                                                                                  
 reshape (Reshape)              (None, 12, 12, 1)    0           ['dense[0][0]']              

In [5]:
def get_direction(action):
    return "csnwe"[action] if action < 5 else None 

In [6]:
def get_unit_future_position(unit, direction):
    to_x = unit.pos.x
    to_y = unit.pos.y

    if direction == "e":
        to_x += 1
    elif direction == "s":
        to_y += 1
    elif direction == "w":
        to_x -= 1
    elif direction == "n":
        to_y -= 1
        
    return to_x, to_y

In [7]:
def is_unit_action_valid(unit, option, actions, player, opponent):
    height, width = game_state.map.width, game_state.map.height
    
    if not unit.can_act():
        return False
    
    # if option == move:
    if option < 5:
        to_x, to_y = get_unit_future_position(unit, get_direction(option))

        # Out of bond
        if to_x < 0 or to_x >= width or to_y < 0 or to_y >= height:
            return False

        to_cell = game_state.map.get_cell(to_x, to_y)
        to_citytile = to_cell.citytile

        # Not citytile and cell already has unit
        if to_citytile is None:
            has_player_unit = to_cell.has_player_unit(player)
            has_opponent_unit = to_cell.has_player_unit(opponent)
            
            if has_player_unit or has_opponent_unit:
                return False

            for action in actions:
                # Move action string is "m {} {}".format(self.id, dir)
                action = action.split(" ")

                if action[0] != "m":
                    continue

                _, player_unit_id, direction = action

                player_unit_to_x = player_unit_to_y = None
                for player_unit in player.units:
                    if player_unit.id != player_unit_id:
                        continue

                    player_unit_to_x, player_unit_to_y = get_unit_future_position(player_unit, direction)
                    break

                if player_unit_to_x is None or player_unit_to_y is None:
                    continue

                if to_x == player_unit_to_x and to_y == player_unit_to_y:
                    return False
                    
        # Opponent citytile
        elif to_citytile.team == opponent.team:
            return False
    #elif option == build_city:
    elif option == 5:
        if not unit.can_build(game_state.map):
            return False
    else: return False
    '''elif option == pillage:
        to_cell = get_cell(to_x, to_y)

        # Not road
        if to_cell.road == 0:
            return False'''

    return True

In [8]:
def is_city_tile_action_valid(city_tile, action, player):
    if not city_tile.can_act():
        return False
    
    #if action == research:
    if action == 6:
        pass
    #elif action == build_worker or action == build_cart:
    elif action == 7:
        owned_units = len(player.units)
        owned_city_tiles = 0
        
        for city in player.cities.values():
            owned_city_tiles += len(city.citytiles)

        if owned_units >= owned_city_tiles:
            return False
    else: return False
        
    return True

In [9]:
def get_best_unit_valid_action(unit, options, actions, player, opponent, i=1):
    if i == len(options):
        return -1
    
    option = np.argsort(options)[-i]
    
    if is_unit_action_valid(unit, option, actions, player, opponent):
        return option
    
    return get_best_unit_valid_action(unit, options, actions, player, opponent, i + 1)

In [10]:
def get_best_city_tile_valid_action(city_tile, options, player, i=1):
    if i == len(options):
        return -1
    
    option = np.argsort(options)[-i]
    
    if is_city_tile_action_valid(city_tile, option, player):
        return option
    
    return get_best_city_tile_valid_action(city_tile, options, player, i + 1)

In [11]:
def get_prediction_actions(y_A, y_B, player, opponent):
    actions = []
    best_options = np.zeros((game_state.map.width, game_state.map.height), dtype=int)

    for unit in player.units:
        unit_y, unit_x = unit.pos.y, unit.pos.x

        options = y_A[unit_y][unit_x] + y_B[unit_y][unit_x]
        
        best_option = get_best_unit_valid_action(unit, options, actions, player, opponent)
        best_options[unit_y, unit_x] = best_option

        if -1 < best_option < 5:
            actions.append(unit.move(get_direction(best_option)))
        elif best_option == 5:
            actions.append(unit.build_city())
            
    for city in player.cities.values():
        for city_tile in city.citytiles:
            city_tile_y, city_tile_x = city_tile.pos.y, city_tile.pos.x
            
            options = y_A[city_tile_y][city_tile_x] + y_B[city_tile_y][city_tile_x]
            
            best_option = get_best_city_tile_valid_action(city_tile, options, player)
            best_options[city_tile_y, city_tile_x] = best_option
        
            if best_option == 6:
                actions.append(city_tile.research())
            elif best_option == 7:
                actions.append(city_tile.build_worker())
    
    return actions, best_options

In [12]:
Last_State = {}
learning_rate = 0.01
gamma = 0.95
epsilon = 1.0
epsilon_final = 0.01
epsilon_decay = 0.995
game_state = None
model = None
last_reward = 0
W = 0
def agent(observation, configuration):
    global game_state, epsilon, model_A, model_B, last_reward, W
    
    ### Do not edit ###
    if observation["step"] == 0:
        game_state = Game()
        game_state._initialize(observation["updates"])
        game_state._update(observation["updates"][2:])
        game_state.id = observation.player
    else:
        game_state._update(observation["updates"])
    

    ### AI Code goes down here! ### 
    player = game_state.players[observation.player]
    opponent = game_state.players[(observation.player + 1) % 2]
    width, height = game_state.map.width, game_state.map.height

    # Get Prediction of actions
    x = get_inputs(game_state)
    y_A = model_A.predict(np.asarray([x]))[0]
    y_B = model_B.predict(np.asarray([x]))[0]
    
    if random.random() < epsilon:
        y_A = np.random.rand(*y_A.shape)
        y_B = np.random.rand(*y_B.shape)
        
    print("eps ",epsilon,end= " | ") 
    actions_A, option_A = get_prediction_actions(y_A, y_A, player, opponent)
    actions_B, option_B = get_prediction_actions(y_B, y_B, player, opponent)
    
    print("Reward: ", observation["reward"])
    
    is_model_A = random.random() < 0.5
    
    if observation.player in Last_State:
        _x, _y_A, _y_B, _player, _option_A, _option_B = Last_State[observation.player]
        state, next_state, reward = _x, x, observation["reward"]
        
        # Reward 
        if reward > last_reward: 
            r = 1
        elif reward < last_reward: 
            r = -1
        else: 
            r = 0
        
        # Double Q-learning update
        
        if is_model_A:
            model = model_A
            _y = _y_A
            y = y_B
            _option = _option_A
            option = option_A
        else:
            model = model_B
            _y = _y_B
            y = y_A
            _option = _option_B
            option = option_B
 
        for i in _player.units:
            Q1 = _y[i.pos.y, i.pos.x][_option[i.pos.y, i.pos.x]]
            Q2 = y[i.pos.y, i.pos.x][option[i.pos.y, i.pos.x]]
            _y[i.pos.y, i.pos.x][_option[i.pos.y, i.pos.x]] += learning_rate * (r + gamma * Q2 - Q1)
        
        states = [state]
        _y_ = [_y]
        
        model.fit(np.asarray(states), np.asarray(_y_), epochs=1, verbose=1)
        
        if epsilon > epsilon_final:
            epsilon *= epsilon_decay
            
    Last_State[observation.player] = [x, y_A, y_B, player, option_A, option_B]
    last_reward = observation["reward"]
    
    return actions_A if is_model_A else actions_B

In [13]:
episodes = 10

# RL training
#sizes = [12,16,24,32]
sizes = [12]

BACKUP_RATIO = 10.0 # %

for size in sizes:
    BASE_FILE_NAME_A = f"model_ddqn_A_{size}_{episodes}.h5"
    BASE_FILE_NAME_B = f"model_ddqn_B_{size}_{episodes}.h5"

    # Initialize the model
    model_A = get_model(size)
    model_B = get_model(size)
    
    try:
        # Change to either final or backup
        prefix = "backup"
        
        model_A.load_weights(f"{prefix}_{BASE_FILE_NAME_A}", by_name=True, skip_mismatch=True)
        model_B.load_weights(f"{prefix}_{BASE_FILE_NAME_B}", by_name=True, skip_mismatch=True)
        print("Weights loaded!")
    except:
        print("Weights not found...")
        
    Last_State = {}
    aux_end = aux_start = 0
    for eps in range(episodes):
        epsilon = 0.2 # Maintaining exploration
        clear_output()
        
        print(f"Elapsed time: {aux_end - aux_start}")
        
        aux_start = time.time()
        print("=== Episode {} ===".format(eps))
        env = make("lux_ai_2021", debug=True, configuration={"annotations": True, "width":size, "height":size})
        steps = env.run(["simple_agent", agent])
        
        aux_end = time.time()
        if eps % int(episodes*(BACKUP_RATIO/100)) == 0:
            model_A.save_weights(f"backup_{BASE_FILE_NAME_A}")
            model_B.save_weights(f"backup_{BASE_FILE_NAME_B}")
        
    # Save the models
    model_A.save_weights(f"final_{BASE_FILE_NAME_A}")
    model_B.save_weights(f"final_{BASE_FILE_NAME_B}")

=== Episode 9 ===
eps  0.2 | Reward:  0
1/1 [==============================] - 0s 13ms/step - loss: 1.2753e-07
eps  0.199 | Reward:  10001
1/1 [==============================] - 0s 10ms/step - loss: 1.5327e-07
eps  0.19800500000000001 | Reward:  10001
1/1 [==============================] - 0s 12ms/step - loss: 1.1803e-08
eps  0.197014975 | Reward:  10001
1/1 [==============================] - 0s 10ms/step - loss: 2.3546e-08
eps  0.19602990012500002 | Reward:  10001
1/1 [==============================] - 0s 18ms/step - loss: 0.1615
eps  0.19504975062437502 | Reward:  10001
1/1 [==============================] - 0s 13ms/step - loss: 2.7509e-04
eps  0.19407450187125314 | Reward:  10001
1/1 [==============================] - 0s 14ms/step - loss: 1.2367e-08
eps  0.19310412936189686 | Reward:  20001
1/1 [==============================] - 0s 13ms/step - loss: 1.3905e-07
eps  0.19213860871508737 | Reward:  20001
1/1 [==============================] - 0s 16ms/step - loss: 1.0971e-05
eps  0.1911

eps  0.13801871219843223 | Reward:  30002
1/1 [==============================] - 0s 11ms/step - loss: 2.4242e-07
eps  0.13732861863744006 | Reward:  30001
1/1 [==============================] - 0s 19ms/step - loss: 1.4823e-07
eps  0.13664197554425286 | Reward:  30001
1/1 [==============================] - 0s 11ms/step - loss: 2.4281e-09
eps  0.1359587656665316 | Reward:  30000
1/1 [==============================] - 0s 20ms/step - loss: 0.0000e+00
eps  0.13527897183819892 | Reward:  30001
1/1 [==============================] - 0s 14ms/step - loss: 8.0416e-08
eps  0.13460257697900793 | Reward:  30000
1/1 [==============================] - 0s 19ms/step - loss: 0.0000e+00
eps  0.13392956409411289 | Reward:  30000
1/1 [==============================] - 0s 13ms/step - loss: 1.5423e-09
eps  0.1332599162736423 | Reward:  30001
1/1 [==============================] - 0s 20ms/step - loss: 8.1280e-08
eps  0.1325936166922741 | Reward:  30001
1/1 [==============================] - 0s 15ms/step - los

eps  0.09572444681866157 | Reward:  60006
1/1 [==============================] - 0s 14ms/step - loss: 8.8296e-05
eps  0.09524582458456826 | Reward:  60006
1/1 [==============================] - 0s 14ms/step - loss: 2.4640e-08
eps  0.09476959546164541 | Reward:  60006
1/1 [==============================] - 0s 19ms/step - loss: 3.6724e-08
eps  0.09429574748433718 | Reward:  60006
1/1 [==============================] - 0s 23ms/step - loss: 3.7092e-08
eps  0.09382426874691549 | Reward:  60006
1/1 [==============================] - 0s 20ms/step - loss: 3.1172e-08
eps  0.09335514740318092 | Reward:  60006
1/1 [==============================] - 0s 16ms/step - loss: 2.6927e-08
eps  0.09288837166616501 | Reward:  60006
1/1 [==============================] - 0s 16ms/step - loss: 3.9790e-08
eps  0.09242392980783419 | Reward:  60006
1/1 [==============================] - 0s 17ms/step - loss: 1.2265e-05
eps  0.09196181015879502 | Reward:  50006
1/1 [==============================] - 0s 13ms/step - 

eps  0.0663907782704471 | Reward:  70007
1/1 [==============================] - 0s 17ms/step - loss: 4.2494e-08
eps  0.06605882437909487 | Reward:  70007
1/1 [==============================] - 0s 19ms/step - loss: 5.3169e-08
eps  0.0657285302571994 | Reward:  70007
1/1 [==============================] - 0s 18ms/step - loss: 4.1951e-08
eps  0.06539988760591339 | Reward:  70007
1/1 [==============================] - 0s 14ms/step - loss: 5.7800e-08
eps  0.06507288816788383 | Reward:  70007
1/1 [==============================] - 0s 10ms/step - loss: 5.4173e-08
eps  0.06474752372704441 | Reward:  70007
1/1 [==============================] - 0s 18ms/step - loss: 3.0508e-08
eps  0.0644237861084092 | Reward:  70007
1/1 [==============================] - 0s 14ms/step - loss: 4.6953e-08
eps  0.06410166717786715 | Reward:  70007
1/1 [==============================] - 0s 17ms/step - loss: 6.0595e-08
eps  0.06378115884197781 | Reward:  70007
1/1 [==============================] - 0s 15ms/step - los

eps  0.04604607898863699 | Reward:  70006
1/1 [==============================] - 0s 12ms/step - loss: 4.1528e-08
eps  0.0458158485936938 | Reward:  70007
1/1 [==============================] - 0s 12ms/step - loss: 6.2772e-07
eps  0.04558676935072533 | Reward:  70007
1/1 [==============================] - 0s 12ms/step - loss: 1.4601e-05
eps  0.045358835503971705 | Reward:  70007
1/1 [==============================] - 0s 13ms/step - loss: 4.5420e-08
eps  0.04513204132645185 | Reward:  70007
1/1 [==============================] - 0s 13ms/step - loss: 3.3271e-08
eps  0.04490638111981959 | Reward:  70007
1/1 [==============================] - 0s 15ms/step - loss: 4.6523e-08
eps  0.04468184921422049 | Reward:  80007
1/1 [==============================] - 0s 10ms/step - loss: 6.4373e-07
eps  0.04445843996814939 | Reward:  80008
1/1 [==============================] - 0s 23ms/step - loss: 8.9487e-07
eps  0.04423614776830864 | Reward:  80008
1/1 [==============================] - 0s 13ms/step - 